In [4]:
%load_ext autoreload
%autoreload 2

from util_0802_2 import *
from myalgorithm_0802_2 import algorithm as algorithm

import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 실험 내용
- 시우님 코드를 탐색하는 가중치를 늘리는 방식으로 개선하는 방식 + SA에서 번들 묶을 때 모든 조합을 사용하는 방식

### 초기 코드 개선 시도

In [5]:
# 기본 데이터셋 추가
data_sizes = [50, 100, 200]
problem_files = []
for data_size in data_sizes:
    problem_files.append(fr'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K{data_size}_1.json')
    problem_files.append(fr'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K{data_size}_2.json')

# stage 데이터 추가
for file_num in range(1, 19):
    problem_file = fr'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_{file_num}.json'

    problem_files.append(problem_file)

results = []
for problem_file in problem_files:
    ## ------------------- 기본 변수 할당 ---------------------------

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    inf = float('inf')

    timelimit = 60

    ## --------------------- 커스텀 코드 시작 ---------------------------

    solution = algorithm(K, ALL_ORDERS, ALL_RIDERS, DIST, timelimit)

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

    print(problem_file)

    results.append((problem_file, checked_solution['avg_cost']))

df = pd.DataFrame(results, columns=['Name', 'Score'])

df.to_csv('8-1_siwoo_changed_30sec_SAAllBundle.csv', index=False, encoding='utf-8-sig')

C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K50_1.json
C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K50_2.json
C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K100_1.json
C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K100_2.json
C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K200_1.json
C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K200_2.json
C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_1.json
C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_2.json
C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_3.json
C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_4.json
C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_5.json
C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_6.json
C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_7.json
C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_8.json
C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_9.json
C:\Users\hsh80\Des

### 결과 분석

In [24]:
base_score_df = pd.read_csv('8-1_siwoo_base.csv')
changed_score_df = pd.read_csv('8-1_siwoo_changed.csv')

In [25]:
score_df = base_score_df.merge(changed_score_df, on='Name')

In [36]:
score_df['Score_x'].sum() - score_df['Score_y'].sum()

747.076666666675

In [53]:
Ks = [50, 50, 100, 100, 200, 200, 100, 100, 200, 200, 300, 300, 100, 100, 200, 200, 300, 300, 100, 100, 200, 200, 300, 300]

score_df['K'] = Ks
score_df['diff'] = score_df['Score_x'] - score_df['Score_y']

score_df2 = score_df.drop('Name', axis=1)


In [55]:
score_df2.groupby('K').mean()

,Score_x,Score_y,diff
K,,,
50,3881.129000,3836.242000,44.887000
100,3761.213625,3732.292375,28.921250
200,3215.962000,3195.181375,20.780625
300,2854.000778,2810.719500,43.281278
